In [58]:
import pandas as pd 
import numpy as np 
import yfinance as yf 
import math
from scipy import stats

In [59]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head()

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank


In [60]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
         ticker = yf.Ticker(stock)
         info = ticker.info
         
         dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
         dividend_rate = info.get("dividendRate", np.nan) 
         payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
         five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
         earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
         dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
    numeric_cols = [
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"        
    ]
    
    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col + 'Normalized'] = 1-((dividend_df[col] - dividend_df[col].min())/(dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col + 'Normalized'] = (dividend_df[col] - dividend_df[col].min())/(dividend_df[col].max() - dividend_df[col].min())
    
    return dividend_df    

In [61]:
tickers_list = tickers['Ticker'].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%)Normalized,Dividend RateNormalized,Payout Ratio(%)Normalized,5 Year Avg Dividend Yield(%)Normalized,Earning Growth(%)Normalized
0,RELIANCE.NS,35.00,5.50,8.960000,44.0,9.6,0.042836,0.028302,0.084725,0.043981,0.125077
1,TCS.NS,189.00,62.00,44.650000,136.0,1.4,0.239636,0.383648,0.474397,0.150463,0.099690
2,HDFCBANK.NS,112.00,11.00,25.150000,67.0,9.1,0.141236,0.062893,0.261491,0.070602,0.123529
3,INFY.NS,275.00,45.00,64.239997,223.0,6.6,0.349537,0.276730,0.688285,0.251157,0.115789
4,ICICIBANK.NS,81.00,11.00,14.880000,99.0,2.0,0.101620,0.062893,0.149361,0.107639,0.101548
5,HINDUNILVR.NS,189.00,43.00,92.790000,155.0,3.6,0.239636,0.264151,1.000000,0.172454,0.106502
6,SBIN.NS,162.00,15.90,17.690000,139.0,2.9,0.205132,0.093711,0.180041,0.153935,0.104334
7,BAJFINANCE.NS,44.00,4.40,14.950000,24.0,21.4,0.054337,0.021384,0.150126,0.020833,0.161610
8,BHARTIARTL.NS,76.00,16.00,24.900000,80.0,88.3,0.095231,0.094340,0.258762,0.085648,0.368731
9,ITC.NS,391.00,15.70,89.080000,391.0,2.5,0.497776,0.092453,0.959493,0.445602,0.103096


In [63]:
weights = {
    "Dividend Yield(%)Normalized": 0.3,
    "Dividend RateNormalized": 0.2,
    "Payout Ratio(%)Normalized": 0.2,
    "5 Year Avg Dividend Yield(%)Normalized": 0.2,
    "Earning Growth(%)Normalized": 0.1
}

dividend_df["Dividend Score"] = (dividend_df[list(weights.keys())] .mul(list(weights.values())).sum(axis=1))
dividend_df


,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%)Normalized,Dividend RateNormalized,Payout Ratio(%)Normalized,5 Year Avg Dividend Yield(%)Normalized,Earning Growth(%)Normalized,Dividend Score
0,RELIANCE.NS,35.00,5.50,8.960000,44.0,9.6,0.042836,0.028302,0.084725,0.043981,0.125077,0.056760
1,TCS.NS,189.00,62.00,44.650000,136.0,1.4,0.239636,0.383648,0.474397,0.150463,0.099690,0.283561
2,HDFCBANK.NS,112.00,11.00,25.150000,67.0,9.1,0.141236,0.062893,0.261491,0.070602,0.123529,0.133721
3,INFY.NS,275.00,45.00,64.239997,223.0,6.6,0.349537,0.276730,0.688285,0.251157,0.115789,0.359675
4,ICICIBANK.NS,81.00,11.00,14.880000,99.0,2.0,0.101620,0.062893,0.149361,0.107639,0.101548,0.104620
5,HINDUNILVR.NS,189.00,43.00,92.790000,155.0,3.6,0.239636,0.264151,1.000000,0.172454,0.106502,0.369862
6,SBIN.NS,162.00,15.90,17.690000,139.0,2.9,0.205132,0.093711,0.180041,0.153935,0.104334,0.157511
7,BAJFINANCE.NS,44.00,4.40,14.950000,24.0,21.4,0.054337,0.021384,0.150126,0.020833,0.161610,0.070931
8,BHARTIARTL.NS,76.00,16.00,24.900000,80.0,88.3,0.095231,0.094340,0.258762,0.085648,0.368731,0.153192
9,ITC.NS,391.00,15.70,89.080000,391.0,2.5,0.497776,0.092453,0.959493,0.445602,0.103096,0.459152


In [64]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%)Normalized,Dividend RateNormalized,Payout Ratio(%)Normalized,5 Year Avg Dividend Yield(%)Normalized,Earning Growth(%)Normalized,Dividend Score
30,COALINDIA.NS,687.0,26.50,63.200000,870.0,-30.8,0.876042,0.160377,0.676930,1.000000,0.000000,0.630274
20,ONGC.NS,784.0,18.25,42.080003,508.0,5.0,1.000000,0.108491,0.446337,0.581019,0.110836,0.538253
42,HEROMOTOCO.NS,225.0,130.00,62.520003,310.0,23.1,0.285641,0.811321,0.669505,0.351852,0.166873,0.468915
9,ITC.NS,391.0,15.70,89.080000,391.0,2.5,0.497776,0.092453,0.959493,0.445602,0.103096,0.459152
22,POWERGRID.NS,512.0,13.50,58.200000,494.0,-5.9,0.652405,0.078616,0.622339,0.564815,0.077090,0.456584
13,HCLTECH.NS,292.0,48.00,86.300004,331.0,-3.9,0.371262,0.295597,0.929141,0.376157,0.083282,0.439886
28,TECHM.NS,279.0,45.00,89.040000,222.0,-4.5,0.354649,0.276730,0.959057,0.250000,0.081424,0.411694
31,BPCL.NS,410.0,15.00,20.130000,541.0,169.3,0.522057,0.088050,0.206682,0.619213,0.619505,0.401357
45,IOC.NS,492.0,8.00,16.620001,699.0,NaN,0.626847,0.044025,0.168359,0.802083,NaN,0.390947
17,WIPRO.NS,416.0,11.00,85.600000,102.0,1.0,0.529724,0.062893,0.921498,0.111111,0.098452,0.387863


In [65]:
dividend_df = dividend_df.head(10)
dividend_df = dividend_df.reset_index()
dividend_df

,index,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%)Normalized,Dividend RateNormalized,Payout Ratio(%)Normalized,5 Year Avg Dividend Yield(%)Normalized,Earning Growth(%)Normalized,Dividend Score
0,30,COALINDIA.NS,687.0,26.50,63.200000,870.0,-30.8,0.876042,0.160377,0.676930,1.000000,0.000000,0.630274
1,20,ONGC.NS,784.0,18.25,42.080003,508.0,5.0,1.000000,0.108491,0.446337,0.581019,0.110836,0.538253
2,42,HEROMOTOCO.NS,225.0,130.00,62.520003,310.0,23.1,0.285641,0.811321,0.669505,0.351852,0.166873,0.468915
3,9,ITC.NS,391.0,15.70,89.080000,391.0,2.5,0.497776,0.092453,0.959493,0.445602,0.103096,0.459152
4,22,POWERGRID.NS,512.0,13.50,58.200000,494.0,-5.9,0.652405,0.078616,0.622339,0.564815,0.077090,0.456584
5,13,HCLTECH.NS,292.0,48.00,86.300004,331.0,-3.9,0.371262,0.295597,0.929141,0.376157,0.083282,0.439886
6,28,TECHM.NS,279.0,45.00,89.040000,222.0,-4.5,0.354649,0.276730,0.959057,0.250000,0.081424,0.411694
7,31,BPCL.NS,410.0,15.00,20.130000,541.0,169.3,0.522057,0.088050,0.206682,0.619213,0.619505,0.401357
8,45,IOC.NS,492.0,8.00,16.620001,699.0,NaN,0.626847,0.044025,0.168359,0.802083,NaN,0.390947
9,17,WIPRO.NS,416.0,11.00,85.600000,102.0,1.0,0.529724,0.062893,0.921498,0.111111,0.098452,0.387863


In [67]:
portfolio_size = int(input("Enter the amount you wanna to inverst:"))
postion_size = portfolio_size/len(dividend_df.index)
print(postion_size)

10000.0


In [70]:
prices = yf.download(dividend_df["Ticker"].tolist(), period="1d", group_by="ticker")

dividend_df["Price"] = dividend_df["Ticker"].apply(
    lambda t: prices[t]["Close"].iloc[-1]
)



/var/folders/hq/_ktb7jds1535nk1wq7r3wpl80000gn/T/ipykernel_53705/3233712776.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(dividend_df["Ticker"].tolist(), period="1d", group_by="ticker")
[*********************100%***********************]  10 of 10 completed


In [71]:
dividend_df['Number of shares can buy'] = dividend_df['Price'].apply(lambda price : math.floor(postion_size/price)) 
dividend_df

,index,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%)Normalized,Dividend RateNormalized,Payout Ratio(%)Normalized,5 Year Avg Dividend Yield(%)Normalized,Earning Growth(%)Normalized,Dividend Score,Price,Number of shares can buy
0,30,COALINDIA.NS,687.0,26.50,63.200000,870.0,-30.8,0.876042,0.160377,0.676930,1.000000,0.000000,0.630274,385.600006,25
1,20,ONGC.NS,784.0,18.25,42.080003,508.0,5.0,1.000000,0.108491,0.446337,0.581019,0.110836,0.538253,232.889999,42
2,42,HEROMOTOCO.NS,225.0,130.00,62.520003,310.0,23.1,0.285641,0.811321,0.669505,0.351852,0.166873,0.468915,5781.000000,1
3,9,ITC.NS,391.0,15.70,89.080000,391.0,2.5,0.497776,0.092453,0.959493,0.445602,0.103096,0.459152,401.049988,24
4,22,POWERGRID.NS,512.0,13.50,58.200000,494.0,-5.9,0.652405,0.078616,0.622339,0.564815,0.077090,0.456584,263.450012,37
5,13,HCLTECH.NS,292.0,48.00,86.300004,331.0,-3.9,0.371262,0.295597,0.929141,0.376157,0.083282,0.439886,1642.400024,6
6,28,TECHM.NS,279.0,45.00,89.040000,222.0,-4.5,0.354649,0.276730,0.959057,0.250000,0.081424,0.411694,1612.500000,6
7,31,BPCL.NS,410.0,15.00,20.130000,541.0,169.3,0.522057,0.088050,0.206682,0.619213,0.619505,0.401357,365.950012,27
8,45,IOC.NS,492.0,8.00,16.620001,699.0,NaN,0.626847,0.044025,0.168359,0.802083,NaN,0.390947,162.600006,61
9,17,WIPRO.NS,416.0,11.00,85.600000,102.0,1.0,0.529724,0.062893,0.921498,0.111111,0.098452,0.387863,264.450012,37
